## 문서 번호 부여 및 파일명 변경

In [ ]:
import os, datetime, re
import pandas as pd

  ### 1. 네이버 드라이브 파일 - 파일명 변환(생략)


In [ ]:
# 1) 네이버 드라이브 파일에 직접 DOC부여
# zip파일이 있으면 다 풀어서 보관
# 엑셀 파일에서 리스트 만들고 DOC번호 부여해서 불러오는게 나을듯

class Test:
    def __init__(self, base_path):
        self.base_path = base_path

    def assign_DocNum_Help(self, start_doc_number):
        """ 동일한 파일명, 확장자, 최종 수정일 기준으로 동일한 DOC 번호 부여 """
        folder_path = self.base_path
        doc_counter = start_doc_number
        seen_files = dict()  # {(파일명, 확장자, 수정일): DOC 번호}

        for root, _, files in os.walk(folder_path):
            for file in sorted(files):
                src_path = os.path.join(root, file)

                # 이미 DOCx_ 형식이면 건너뜀
                if re.match(r'DOC\d+_', file):
                    continue

                file_name, ext = os.path.splitext(file)
                mod_time = os.path.getmtime(src_path)
                file_key = (file_name, ext, mod_time) # (파일명, 확장자, 최종 업로드일)

                if file_key in seen_files:
                    doc_num = seen_files[file_key]
                else:
                    doc_num = doc_counter
                    seen_files[file_key] = doc_num
                    doc_counter += 1

                new_file_name = f"DOC{doc_num:05d}_{file}"
                new_path = os.path.join(root, new_file_name)

                os.rename(src_path, new_path)

In [ ]:
# 1) 네이버 드라이브 파일에 직접 DOC부여

# zip파일이 있으면 다 풀어서 보관
# 엑셀 파일에서 리스트 만들고 DOC번호 부여해서 불러오는게 나을듯

def assign_document_numbers(base_folder, start_doc_number):
    """ 기존 파일에 DOC1, DOC2 등의 문서 번호를 부여 """
    doc_counter = start_doc_number
    for root, _, files in os.walk(base_folder):
        for file in sorted(files):  # 파일명을 정렬하여 일관된 번호 부여
            src_path = os.path.join(root, file)

            # 기존 파일명이 DOCx_ 형식이면 건너뜀
            if re.match(r'DOC\d+_', file):
                continue

            new_file_name = f"DOC{doc_counter}_{file}"
            new_path = os.path.join(root, new_file_name)
            os.rename(src_path, new_path)
            doc_counter += 1

def assign_DocNum_all_product_folders(base_directory, start_doc_number):
    for folder in os.listdir(base_directory):
        folder_path = os.path.join(base_directory, folder)

        # BrandLine 폴더 내 BrandName 폴더 찾기
        if os.path.isdir(folder_path):
            for file_name in os.listdir(folder_path):
                file_name_path = os.path.join(folder_path, file_name)
                print(f"Assigning: {file_name}")
                assign_document_numbers(file_name_path, start_doc_number)

naver_base_directory = 'C:/Users/LGCARE/Desktop/01. 빌리프_컨텐츠아카이빙/01. 빌리프 드라이브/01. 누끼컷 모음_문서 번호 부여'
assign_DocNum_all_product_folders(naver_base_directory, 1)

In [ ]:
# 2) 문서 번호 부여 후 데이터프레임 생성

# 문서 번호 | (구)파일명 | 확장자 | 최종 업로드일 | 용량(MB) | 폴더 경로 
def generate_DocNum_dataframe(naver_drive_directory, start_doc_num):
  """
  지정된 디렉토리 내 모든 파일에 대해 문서 번호를 부여하고
  파일 정보를 포함한 데이터프레임을 반환
  Parameters:
  - target_directory (str): 검색할 폴더 경로
  - start_doc_num (int): 문서 번호 시작 값 (예: 1이면 DOC00001부터 시작)
  Returns:
  - pandas.DataFrame: 열 - 문서 번호, (구)파일명, 확장자, 최종 업로드일, 용량, 폴더 경로
  """
  allowed_exts = {'.jpg', '.psd', '.png', '.ai', '.mp4', '.pdf', '.ssg', '.gif', '.fig'}
  data = []
  doc_counter = start_doc_num
  for root, _, files in os.walk(naver_drive_directory):
      for file in sorted(files):
          file_path = os.path.join(root, file)
          file_name, file_ext = os.path.splitext(file)
          file_ext = file_ext.lower()
          if file_ext not in allowed_exts :
              continue
          mod_time = datetime.datetime.fromtimestamp(os.path.getmtime(file_path)).strftime('%Y-%m-%d')
          file_size_kb = os.path.getsize(file_path) / (1024 ** 2)
          file_size_str = f"{file_size_kb:.2f} MB"
          data.append({
              '(구)파일명': file,
              '확장자': file_ext,
              '최종 업로드일': mod_time,
              '용량(MB)': file_size_str,
              '폴더 경로': root
          })
  
  df = pd.DataFrame(data)
  df = df.drop_duplicates(subset='(구)파일명', keep= 'first').reset_index(drop=True)
  df.insert(0, '문서 번호', [f'DOC{num:05}' for num in range(start_doc_num, start_doc_num + len(df))])
  print(f"✅ 총 {len(df)}개의 파일이 문서 번호와 함께 데이터프레임으로 저장되었습니다.")
  return df

  ### 2. 파일 트리 내 파일명 변경


In [ ]:
def rename_files(base_folder, product_name):
    """ 문서번호_제품명_용량_상위폴더명_최종수정일 형식으로 변경 """

    def extract_volume_info(file_name):
        """ 파일명에서 '100ml', '200g' 같은 정보를 추출 """
        match = re.search(r'(\d+(?:ml|g))', file_name)
        return match.group(1) if match else ""

    file_dates = {}  # 중복 방지용 딕셔너리

    for root, _, files in os.walk(base_folder):
        folder_name = os.path.basename(root)

        for file in sorted(files):
            src_path = os.path.join(root, file)
            mod_time = datetime.datetime.fromtimestamp(os.path.getmtime(src_path)).strftime('%Y%m%d')

            # 기존 파일명에서 DOCx_ 문서번호 추출
            doc_match = re.match(r'(DOC\d+)_', file)
            doc_number = doc_match.group(1) if doc_match else "DOC0"

            # 파일명에서 용량 정보 추출
            volume_info = extract_volume_info(file)
            volume_suffix = f"_{volume_info}" if volume_info else ""

            # 새로운 파일명 생성
            new_name = f"{doc_number}_{product_name}_{volume_suffix}_{folder_name}_{mod_time}"

            ext = os.path.splitext(file)[1]
            count = file_dates.get(new_name, 0) + 1
            file_dates[new_name] = count
            new_file_name = f"{new_name}_{count}{ext}" if count > 1 else f"{new_name}{ext}"

            new_path = os.path.join(root, new_file_name)
            os.rename(src_path, new_path)

def rename_all_product_folders(base_directory):
    for brand_line in os.listdir(base_directory):
        brand_line_path = os.path.join(base_directory, brand_line)

        # "0_BrandAsset" 폴더는 건너뛰기
        if brand_line == "0_BrandAsset":
            print(f"Skipping brand asset folder: {brand_line}")
            continue

        # BrandLine 폴더 내 BrandName 폴더 찾기
        if os.path.isdir(brand_line_path):
            for product_name in os.listdir(brand_line_path):
                product_name_path = os.path.join(brand_line_path, product_name)
                print(f"Renaming: {product_name}")
                rename_files(product_name_path, product_name)

In [ ]:
# 이미 존재했던 폴더에 대해서도 파일명 변경
google_base_directory = "C:/Users/LGCARE/Desktop/01. 빌리프_컨텐츠아카이빙/Belif_작업중_문서번호" 
rename_all_product_folders(google_base_directory)

Skipping brand asset folder: 0_BrandAsset
Renaming: BarrierEmulsion
Renaming: BarrierToner
Renaming: Bomb
Renaming: EyeSerum
Renaming: VCream
Renaming: VCreamAD
Renaming: VEyeSerum
Renaming: WaterEssence
Renaming: AquaBomb
Renaming: BrighteningAmpouleSerum
Renaming: CoolingSunstick
Renaming: EyeGel
Renaming: JellyCleanser
Renaming: Mist
Renaming: RefiningAmpouleSerum
Renaming: SleepingMask
Renaming: SmartCleansingOilBalm
Renaming: SunSerum
Renaming: SunSerumAD
Renaming: VitaWaterCream
Renaming: BergamotHerbalExtractToner
Renaming: ClassicCreamUltimate
Renaming: ClassicEssenceIncrement
Renaming: CreamyMoisturizerDeepMoist
Renaming: EucalyptusHerbalExtractToner
Renaming: HungarianWaterEssence
Renaming: HungarianWaterEssenceMoistureBinder
Renaming: MilkyMoisturizerHydraBalancing
Renaming: Numero10Essence
Renaming: Numero10Mist
Renaming: OilControlMoisturizerFresh
Renaming: WitchHazelHerbalExtractToner
Renaming: All-in-oneFacialLotion
Renaming: FaicialBubbleWash
Renaming: MildJellyShampoo


In [2]:
# test필요
# def renameContent(self, mode='product'):
#     """
#     제품 / 기획세트 통합 리네이밍
#     mode: 'product' or 'edition'
#     """
#     def extract_volume_info(file_name):
#         match = re.search(r'(\d+(?:ml|g))', file_name)
#         return match.group(1) if match else ""

#     def extract_country_keyword(file_name):
#         keywords = ['국내', '국내용', '중국', '중국용', '국내중국겸용', '일본', '일본용',
#                     '미국', '북미', '북미용', '유럽', '유럽용', '베트남', '베트남용',
#                     '동남아', '동남아시아', '동남아시아용']
#         return ''.join([f"_{kw}" for kw in keywords if kw in file_name])

#     file_dates = {}

#     for product_line in os.listdir(self.base_path):
#         product_line_path = os.path.join(self.base_path, product_line)

#         if product_line == "0_BrandAsset_브랜드자산":
#             print(f"Skipping brand asset folder: {product_line}")
#             continue

#         if mode == 'product' and product_line == '1_EditionSet_기획세트':
#             continue
#         if mode == 'edition' and product_line != '1_EditionSet_기획세트':
#             continue

#         if mode == 'product':
#             for product_name in os.listdir(product_line_path):
#                 product_name_path = os.path.join(product_line_path, product_name)
#                 if os.path.isdir(product_name_path):
#                     for product_type in os.listdir(product_name_path):
#                         type_path = os.path.join(product_name_path, product_type)
#                         self.__renameContentHelp(type_path, product_name, extract_volume_info, extract_country_keyword, file_dates)
#                         print(f"Renaming: {product_name}")
#         else:
#             for year in os.listdir(product_line_path):
#                 year_path = os.path.join(product_line_path, year)
#                 for channel_or_season in os.listdir(year_path):
#                     channel_path = os.path.join(year_path, channel_or_season)
#                     for edition_type in os.listdir(channel_path):
#                         edition_path = os.path.join(channel_path, edition_type)
#                         self.__renameContentHelp(edition_path, edition_type, extract_volume_info, extract_country_keyword, file_dates)
#                         print(f"Renaming: {edition_type}")


# def __renameContentHelp(self, type_path, item_name, extract_volume_info, extract_country_keyword, file_dates):
#     for root, _, files in os.walk(type_path):
#         folder_name = os.path.basename(root)
#         for file in sorted(files):
#             src_path = os.path.join(root, file)
#             mod_time = datetime.datetime.fromtimestamp(os.path.getmtime(src_path)).strftime('%Y%m%d')
#             doc_match = re.match(r'(DOC\d+)_', file)
#             doc_number = doc_match.group(1) if doc_match else ""
#             volume_info = extract_volume_info(file)
#             volume_suffix = f"_{volume_info}" if volume_info else ""
#             country_suffix = extract_country_keyword(file)

#             new_name = f"{doc_number}_{item_name}{volume_suffix}_{folder_name}_{mod_time}{country_suffix}"
#             ext = os.path.splitext(file)[1]
#             count = file_dates.get(new_name, 0) + 1
#             file_dates[new_name] = count
#             new_file_name = f"{new_name}_{count}{ext}" if count > 1 else f"{new_name}{ext}"

#             new_path = os.path.join(root, new_file_name)
#             os.rename(src_path, new_path)